# Scraping & Cleaning for Question 4 and 5
## Import Statements and functions

In [43]:
# import statements
import requests
import re
import json
import time
from bs4 import BeautifulSoup as BS

In [44]:
def scrape(url):
    # need header to scrape from wikipedia
    headers = {
        'User-Agent': 'SchoolProjectBot/1.0 (r1043412@student.thomasmore.be) Python-Requests/2.31.0'
    }
    page = requests.get(url, headers=headers)
    
    time.sleep(2)
    
    return BS(page.text, "html")

In [45]:
def get_rider_year(url):
    soup = scrape(url)

    rider_year = [
        {'rider':row.find('div', class_='riderCont').get_text(strip=True),
          'year': row.find('div', class_='year').get_text(strip=True),
          'link': row.find('div', class_='riderCont').find('a')['href']}

        for row in soup.find('ul', class_='palmares').select('li')[1:-2]
    ]

    return rider_year

In [ ]:
def add_birthyears(dict):
    for rider in dict:
        soup = scrape('https://www.procyclingstats.com/' + rider['link'])

        # debug
        # print(soup.find('div', class_='borderbox left w65'))

        rider.update({'birthyear': soup.find('div', class_='borderbox left w65').find_all('ul', class_='list')[1].select('div')[3].get_text(strip=True)})

    return dict

In [47]:
def get_countries(url):
    soup = scrape(url)

    countries = [
        {'year': row.find('div', class_='year').get_text(strip=True),
         'countries': (row.find_all('div', class_='riderCont')[0].select('span')[1]['class'][1],
         row.find_all('div', class_='riderCont')[1].select('span')[1]['class'][1],
         row.find_all('div', class_='riderCont')[2].select('span')[1]['class'][1])}

        for row in soup.find('ul', class_='palmares').select('li')[1:-2]
    ]

    return countries

In [48]:
def write_json(dict, list):
    with open(f'data/ages.json', 'r+') as file:
        # load the entire json file into memory
        file_json = json.load(file)

        # update the json with the python dictionary
        file_json[list].append(dict)

        # put cursor at beginning of file and overwrite it with the dictionary in memory
        file.seek(0)
        json.dump(file_json, file, indent=4)

## Function Calls

In [ ]:
rider_dict = get_rider_year('https://www.procyclingstats.com/race.php?race=1049&p=results&s=palmares&gctype=4&filter=Filter')
add_birthyears(rider_dict)

write_json(rider_dict, 'victors')

countries_list = get_countries('https://www.procyclingstats.com/race.php?race=1049&p=results&s=palmares&gctype=4&filter=Filter')

write_json(countries_list, 'country')